In [252]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import KNNImputer
from geopy.distance import geodesic

# Classificadores:
from sklearn.neighbors import NearestNeighbors

## 1) Loading Data

In [253]:
df_meteorologia = pd.read_csv("..\Dados\Views\dados_meteorologicos.csv", index_col=0)
df_produto_agricola = pd.read_csv(R"..\Dados\Tabela_final\dados_producao_agricola.csv", index_col=0)
df_municipios_sertao = pd.read_csv(r"..\Dados\Views\municipios_sertao.csv", index_col=0)
df_transporte = pd.read_csv(r"..\Dados\Views\custo_de_transporte.csv", index_col=0)
df_recursos_hidricos = pd.read_csv(r"..\Dados\Views\recursos_hidricos.csv", index_col=0)
df_solo = pd.read_csv(r"..\Dados\Views\solos_municipios.csv", index_col=0)

In [254]:
df_classificador = df_municipios_sertao[["IBGE7", "NOME", "LATITUDE", "LONGITUDE"]].copy()

In [255]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE
0,2700300,ARAPIRACA,-9.75487,-36.6615
1,2700706,BATALHA,-9.67420,-37.1330
2,2700904,BELO MONTE,-9.82272,-37.2770
3,2701209,CACIMBINHAS,-9.40121,-36.9911
4,2701605,CANAPI,-9.11932,-37.5967


In [256]:
# Merges com o dataframe do classificador:
df_classificador = df_classificador.merge(df_transporte, on="IBGE7", how="left")
df_classificador = df_classificador.merge(df_meteorologia, on="IBGE7", how="left")
df_classificador = df_classificador.merge(df_recursos_hidricos[["IBGE7", "AREA_IRRIGADA_TOT"]], on="IBGE7", how="left")

# Merge do dataframe de produtos agrícolas com o de municípios do sertão:
df_produto_agricola = df_produto_agricola.merge(df_municipios_sertao[["IBGE7", "NOME"]], how="inner", on="NOME")

In [257]:
df_solo.head()

,IBGE7,SOLO,AREA_TOTAL
0,2207959,LAd - Latossolos Amarelos Distroficos,64739.067171
1,2207959,PVAe - Argissolos Vermelho-Amarelos Eutroficos,4865.805456
2,2207959,RQo - Neossolos Quartzarenicos Orticos,24660.252674
3,2207934,PVAe - Argissolos Vermelho-Amarelos Eutroficos,23145.966454
4,2207934,RQo - Neossolos Quartzarenicos Orticos,65225.223645


In [258]:
df_produto_agricola["PRODUTO"].unique()

array(['ALGODAO HERBACEO (EM CAROCO)', 'AMENDOIM (EM CASCA)', 'BANANA ',
       'CASTANHA DE CAJU', 'FAVA (EM GRAO)', 'FEIJAO (EM GRAO)',
       'MANDIOCA', 'MANGA', 'MILHO (EM GRAO)', 'TOMATE',
       'CAFE (EM GRAO) ARABICA', 'CAFE (EM GRAO) TOTAL', 'MELANCIA',
       'MELAO', 'BATATA-DOCE', 'BATATA-INGLESA',
       'CAFE (EM GRAO) CANEPHORA', 'URUCUM ', 'CACAU (EM AMENDOA)',
       'SOJA (EM GRAO)', 'SORGO (EM GRAO)', 'MAMONA ', 'UVA',
       'PIMENTA-DO-REINO', 'TRIGO (EM GRAO)', 'GUARANA '], dtype=object)

In [259]:
# Obtendo o produto, respectivamente, de maior Valor e maior rendimento.
rendimentos_max = df_produto_agricola.groupby(["NOME"]).agg({"VALOR_PROD":"max"}).reset_index()
df_agro = rendimentos_max.merge(df_produto_agricola[["NOME", "IBGE7", "REND_MEDIO", "PRODUTO", "VALOR_PROD"]], on=["NOME", "VALOR_PROD"], how="inner")
df_agro = df_agro.drop_duplicates(["NOME", "VALOR_PROD", "PRODUTO"])
max = df_agro.groupby("NOME")["REND_MEDIO"].idxmax()
df_agro_max = df_agro.loc[max].reset_index(drop=True)
df_agro_max.drop(columns=["NOME"], inplace=True)
df_classificador = df_classificador.merge(df_agro_max, on="IBGE7", how="inner")
df_classificador.drop(columns=["REND_MEDIO", "VALOR_PROD", "LATITUDE", "LONGITUDE"], inplace=True)
df_classificador.rename(columns={"PRODUTO": "PRODUTO_MAIOR_VALOR"},inplace=True)

In [260]:
# Obtendo solo de maior área dentro do município
df_solo_esparsa = df_solo.pivot_table(index="IBGE7", columns="SOLO", values="AREA_TOTAL").fillna(value=0).reset_index()
df_classificador = df_classificador.merge(df_solo_esparsa, how="inner", on="IBGE7")

## 2) Identificar vizinhos Lógicos utilizando Nearest Neighbours

### 2.1) Pré-processamento

In [261]:
X = df_classificador.drop(columns=["IBGE7", "NOME", "PRODUTO_MAIOR_VALOR"])

In [262]:
cat_columns = df_solo["SOLO"].unique()
colunas_num = [i for i in X.columns if i not in cat_columns]
colunas_log = [i for i in X.columns if (i not in cat_columns)&(i not in ["LATITUDE", "LONGITUDE"])]

In [263]:
# Aplicando log transformation para conter a variabilidade dos dados:
log_transformation = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
X[colunas_log] = log_transformation.transform(X[colunas_log])

In [264]:
# Aplicando MinMax scaler para conter o range dos dados:
min_max = MinMaxScaler()
colunas_totais = X.columns
X[colunas_totais] = min_max.fit_transform(X[colunas_totais])

In [265]:
imputer = KNNImputer(n_neighbors=5)
X_imputado = pd.DataFrame(imputer.fit_transform(X))
X_imputado.columns = X.columns

In [280]:
X_imputado.head()

,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,AGUA,CXbd - Cambissolos Haplicos Tb Distroficos,CXbe - Cambissolos Haplicos Tb Eutroficos,...,RLe - Neossolos Litolicos Eutroficos,RQo - Neossolos Quartzarenicos Orticos,RRe - Neossolos Regoliticos Eutroficos,RYve - Neossolos Fluvicos Ta Eutroficos,SNo - Planossolos Natricos Orticos,SXe - Planossolos Haplicos Eutroficos,TCo - Luvissolos Cromicos Orticos,TCp - Luvissolos Cromicos Palicos,VEo - Vertissolos Ebanicos Orticos,VXo - Vertissolos Haplicos Orticos
0,0.295598,0.819860,0.661136,0.647071,0.410457,0.675412,0.538109,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000006,0.0,0.0,0.0,0.0
1,0.324910,0.724461,0.770893,0.770942,0.382409,0.215853,0.000000,0.000000,0.0,0.0,...,0.052826,0.0,0.000000,0.0,0.0,0.017522,0.0,0.0,0.0,0.0
2,0.320801,0.724461,0.770893,0.770942,0.382409,0.215853,0.000000,0.009459,0.0,0.0,...,0.064447,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.338606,0.367246,0.859949,0.672902,0.292261,0.706131,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.015099,0.0,0.0,0.078041,0.0,0.0,0.0,0.0
4,0.389055,0.649550,0.812645,0.716960,0.587898,0.630163,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.307012,0.0,0.0,0.093297,0.0,0.0,0.0,0.0


### 2.2) Aplicando Nearest Neighbours

In [267]:
k_neighburs = 5 
knn = NearestNeighbors(n_neighbors=k_neighburs, n_jobs=-1, metric="cosine")
knn.fit(X_imputado)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [268]:
distance, neighbours_indices = knn.kneighbors(X_imputado, n_neighbors=k_neighburs+1)

#### 2.2.1) ITAPICURU

In [295]:
# Escolha do município a ser avaliado
nome_potencial_mun = "ITAPICURU"

In [296]:
# Selecionandos vizinhos de ITAPICURU:
index_nn = df_classificador[df_classificador["NOME"] == nome_potencial_mun].index[0]
distancias_corrigidas = distance[index_nn][1:]
neighbours_indices_corrigidos = neighbours_indices[index_nn][1:]

In [297]:
# Vizinhos lógicos de ITAPICURU:
colunas_avaliadoras = ["IBGE7","NOME"] + colunas_num + ["PRODUTO_MAIOR_VALOR"]
vizinhos = df_classificador.iloc[neighbours_indices_corrigidos][colunas_avaliadoras]
vizinhos

,IBGE7,NOME,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR
146,2913705,INHAMBUPE,23560.574098,729.771173,1066.567575,25.173941,2.634640,337.79,1770.04332,MILHO (EM GRAO)
257,2926608,RIBEIRA DO POMBAL,24140.726560,778.182862,1034.162314,25.419589,2.172381,182.00,509.00000,MILHO (EM GRAO)
256,2926509,RIBEIRA DO AMPARO,23986.176670,778.182862,1034.162314,25.419589,2.172381,182.00,1545.74325,MELAO
219,2922904,NOVA SOURE,23907.007690,778.182862,1034.162314,25.419589,2.172381,182.00,497.00000,MELANCIA
105,2907905,CIPO,23988.945974,778.182862,1034.162314,25.419589,2.172381,182.00,219.00000,TOMATE


In [281]:
# Calculo do rendimento por área para avaliar se há subexploração:
df_produto_agricola["REND_POR_AREA_PLANTADA"] = df_produto_agricola["REND_MEDIO"] / df_produto_agricola["AREA_PLANTADA"]

In [285]:
# Produtos produzidos em ITAPICURO:
df_itapicuru = df_produto_agricola[df_produto_agricola["NOME"] == nome_potencial_mun].sort_values("VALOR_PROD", ascending=False)
df_itapicuru

,NOME,PRODUTO,AREA_PLANTADA,AREA_COLHIDA,REND_MEDIO,VALOR_PROD,IBGE7,REND_POR_AREA_PLANTADA
3504,ITAPICURU,MILHO (EM GRAO),19333.333333,19333.333333,3689.000000,70163.666667,2916500,0.190810
3501,ITAPICURU,MANDIOCA,6266.666667,4133.333333,9092.666667,28589.000000,2916500,1.450957
3503,ITAPICURU,MELAO,223.666667,223.666667,21234.666667,4674.666667,2916500,94.938897
3499,ITAPICURU,CASTANHA DE CAJU,1295.666667,1295.666667,310.000000,1535.000000,2916500,0.239259
3500,ITAPICURU,FEIJAO (EM GRAO),1006.666667,1006.666667,270.666667,1241.333333,2916500,0.268874
3502,ITAPICURU,MELANCIA,133.666667,133.666667,8848.333333,880.333333,2916500,66.197007
3498,ITAPICURU,BANANA,20.000000,20.000000,13433.333333,569.666667,2916500,671.666667
3505,ITAPICURU,SOJA (EM GRAO),51.666667,51.666667,2995.000000,309.000000,2916500,57.967742


In [288]:
# Solos que podemos encontrar em ITAPICURU:
df_solo[df_solo["IBGE7"].isin([2916500])].merge(df_municipios_sertao[["NOME", "UF", "IBGE7"]], on="IBGE7", how="inner")

,IBGE7,SOLO,AREA_TOTAL,NOME,UF
0,2916500,LAd - Latossolos Amarelos Distroficos,18032.365183,ITAPICURU,BA
1,2916500,PVAd - Argissolos Vermelho-Amarelos Distroficos,46579.916148,ITAPICURU,BA
2,2916500,RQo - Neossolos Quartzarenicos Orticos,86920.869936,ITAPICURU,BA
3,2916500,SNo - Planossolos Natricos Orticos,15514.030117,ITAPICURU,BA


In [298]:
# Rendimento por área dos vizinhos lógicos de ITAPICURU:
df_agrupado = df_produto_agricola[df_produto_agricola["IBGE7"].isin(vizinhos["IBGE7"].values)]

# Soma do Valor produzido dos produtos cultivados pelos vizinhos:
df_agrupado.groupby("PRODUTO").agg({"VALOR_PROD":"sum"}).sort_values("VALOR_PROD", ascending=False)

,VALOR_PROD
PRODUTO,
MILHO (EM GRAO),49954.666667
MELAO,40735.666667
MELANCIA,10719.000000
MANDIOCA,10472.000000
FEIJAO (EM GRAO),7654.000000
CASTANHA DE CAJU,3121.333333
BANANA,2085.000000
AMENDOIM (EM CASCA),637.666667
TOMATE,371.333333


In [291]:
# Solos dos vizinhos lógicos identificados:
df_municipios_sertao[["NOME", "UF", "IBGE7"]].merge(df_solo[df_solo["IBGE7"].isin(vizinhos["IBGE7"].values)], on="IBGE7", how="inner")

,NOME,UF,IBGE7,SOLO,AREA_TOTAL
0,CIPO,BA,2907905,LAd - Latossolos Amarelos Distroficos,11816.751307
1,CIPO,BA,2907905,RQo - Neossolos Quartzarenicos Orticos,2501.373861
2,INHAMBUPE,BA,2913705,LAd - Latossolos Amarelos Distroficos,63582.129104
3,INHAMBUPE,BA,2913705,PVAd - Argissolos Vermelho-Amarelos Distroficos,26682.523981
4,INHAMBUPE,BA,2913705,PVAe - Argissolos Vermelho-Amarelos Eutroficos,39266.429269
5,NOVA SOURE,BA,2922904,LAd - Latossolos Amarelos Distroficos,73498.034469
6,NOVA SOURE,BA,2922904,PVAe - Argissolos Vermelho-Amarelos Eutroficos,25392.652704
7,RIBEIRA DO AMPARO,BA,2926509,LAd - Latossolos Amarelos Distroficos,3967.166293
8,RIBEIRA DO AMPARO,BA,2926509,RQo - Neossolos Quartzarenicos Orticos,61605.365659
9,RIBEIRA DO AMPARO,BA,2926509,SNo - Planossolos Natricos Orticos,1333.137921


## 3) Similaridade Geográfica a partir da distância

In [309]:
# Função para calcular a distância em KM, entre duas coordenadas geográficas:
def distancia_circular(df_municipios_sertao, threhold, municipio_start):
    dicionario_distancias = {}
    lista_distancias = []
    municipio_escolhido = df_municipios_sertao[df_municipios_sertao["NOME"] == municipio_start][["LATITUDE", "LONGITUDE"]]
    for municipio_qualquer, row2 in df_municipios_sertao[["LATITUDE", "LONGITUDE"]].iterrows():
        nome = df_municipios_sertao.loc[municipio_qualquer, "NOME"]
        if municipio_start != nome:
            distancia = geodesic(municipio_escolhido.values, row2.values).kilometers
            if distancia <= threhold:
                lista_distancias.append((nome, distancia))
    
        #dicionario_distancias[df_municipios_sertao.loc[municipio_qualquer, "NOME"]] = lista_distancias
    return pd.DataFrame(lista_distancias, columns=["MUNICIPIOS", "DISTÂNCIAS"])

In [310]:
# Escolha do município a ser avaliado
nome_potencial_mun = "ITAPICURU"
criterio_distancia = 50

In [312]:
# Calculo das distâncias deste município até otros:
distancia_circular(df_municipios_sertao, threhold=criterio_distancia, municipio_start=nome_potencial_mun)

,MUNICIPIOS,DISTÂNCIAS
0,CIPO,39.141885
1,CRISOPOLIS,23.277050
2,NOVA SOURE,29.698246
3,OLINDINA,13.005937
4,RIBEIRA DO AMPARO,36.578826
5,TOBIAS BARRETO,28.573107
